In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import kcgof.cgoftest as cgof
import kcgof.cdensity as cd
import kcgof.util as util
import kcgof.kernel as kernel
import scipy
from scipy.integrate import quad
import numpy as np
import torch.distributions as dists
import torch

In [4]:
seed = 1
n = 10
dx = 1
dy = 1
alpha = 0.05

# Zheng's method

## Null

In [155]:
ols = cd.CDGaussianOLS(torch.ones(dx), c=0, variance=1.0)
zheng_test = cgof.ZhengKLTest(ols, alpha)
source = ols.get_condsource()
X = torch.randn(n, dx)
Y = source.cond_pair_sample(X, seed)
zheng_test.perform_test(X, Y)

KeyboardInterrupt: 

## Alternative

In [153]:
slope = 2.
bias = 10.
ols = cd.CDGaussianOLS(slope*torch.ones(dx), torch.tensor(bias), torch.ones(1))
zheng_test = cgof.ZhengKLTest(ols, alpha)
source = ols.get_condsource()
X = torch.randn(n, dx)
Y = source.cond_pair_sample(X, seed)
zheng_test.perform_test(X, Y)

{'alpha': 0.05,
 'pvalue': 7.908618302243653e-06,
 'test_stat': 4.316988541432568,
 'h0_rejected': True,
 'time_secs': 0.9554009437561035}

# MMD

In [20]:
import freqopttest.tst as tst
import freqopttest.data as fdata

seed = 13
n = 300
dx = 2
dy = 1
alpha = 0.05

## Null

In [16]:
ols = cd.CDGaussianOLS(torch.ones(dx), c=0, variance=1.0)
with util.TorchSeedContext(seed=seed):
    X = torch.randn(n, dx)
source = ols.get_condsource()
Y = source.cond_pair_sample(X, seed)
med_x = util.meddistance(X.numpy(), 1000)
med_y = util.meddistance(Y.numpy(), 1000)
k = kernel.PTKGauss(med_x).numpy()
l = kernel.PTKGauss(med_y).numpy()

mmdtest = cgof.MMDTest(ols, k, l, alpha=alpha, seed=seed)

In [19]:
mmdtest.perform_test(X, Y)

{'alpha': 0.05,
 'pvalue': 1.0,
 'test_stat': -0.004018268359810451,
 'h0_rejected': False,
 'list_permuted_mmd2': array([-2.65220004e-03, -1.87118956e-03,  1.84699868e-03,  6.28832219e-05,
        -2.47154142e-03,  1.68310601e-03,  1.14280675e-03,  3.08733103e-04,
        -3.13980630e-04, -1.06089308e-03,  1.34430306e-03,  3.75620558e-03,
         1.54699693e-03,  1.05074641e-03,  1.02139510e-03, -3.16077463e-03,
        -2.14162225e-03, -2.09376306e-03, -2.55315026e-03, -7.12988608e-04,
         2.29101345e-03,  3.62318841e-04, -1.52898986e-03, -9.60515085e-04,
        -3.56722931e-04, -2.87962915e-03, -1.46323248e-03, -2.72921457e-03,
         5.55214430e-04,  5.70695722e-05,  3.50423721e-03, -1.00978958e-03,
         5.36771878e-04,  3.81022680e-04, -1.29929627e-03, -7.74848453e-04,
        -2.29075216e-03, -3.21207410e-03,  1.10544262e-03,  1.89076435e-03,
        -3.07021669e-03,  2.70987929e-03, -1.52356814e-03,  1.69410187e-03,
        -1.72867893e-03, -1.77000592e-03, -2.95847